In [57]:
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np

In [58]:
# Genera una serie di circuiti per creare una sequenza di qubit nello stato (0 1 + -) a secondo del bit e della base
def codifica_messaggio(bits, basi):
    messaggio = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if basi[i] == 0: # Prepara il qubit in base Z
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepara il qubit in base X
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        messaggio.append(qc)
    return messaggio

In [59]:
def misura_messaggio(messaggio, basi):
    backend = Aer.get_backend('aer_simulator')
    misurazioni = []
    for q in range (n):
        if basi[q] == 0: # misura in base z
            messaggio[q].measure(0,0)
        if basi[q] == 1: # misura in base X
            messaggio[q].h(0)
            messaggio[q].measure(0,0)
        aer_sim = Aer.get_backend('aer_simulator')
        result = aer_sim.run(messaggio[q], shot=1, memory=True).result()
        misura_bit = int(result.get_memory()[0])
        misurazioni.append(misura_bit)
    return misurazioni

In [60]:
'''
Se Bob ha misurato un bit nella stessa base in cui lo ha preparato Alice, significa che il valore in bob_misura
corrisponderà al valore in alice_bits e possono usare quel bit come parte della loro chiave.
Se hanno misurato su basi diverse, il risultato di Bob è casuale ed entrambi buttano via quel bit.

Ecco una funzione rimozione_differenze che fa questo.
'''
def rimozione_differenze(a_basi, b_basi, bits):
    bit_buoni = []
    for q in range(n):
        if a_basi[q] == b_basi[q]:
            bit_buoni.append(bits[q])
    return bit_buoni

In [61]:
# Questa funzione ritorna un array di bit selezionati dalla chiave, togliendoli dalla stessa.

# np.mod(a,b) calcola il resto della divisione a/b
# v = arr.pop(i) rimuove l'elemento iesimo dall'array arr e lo copia in v 

def selezione_bits(bits, selezione):
    selezionati = []
    for i in selezione:
        # con np.mod si fa in modo che i non vada fuori range
        i = np.mod(i, len(bits))
        selezionati.append(bits.pop(i))
    return selezionati

In [62]:
np.random.seed(seed = 1)
n = 10

## Step 1
# Alice genera un array di n bit
alice_bits = randint(2, size = n)
print('Alice bits ', alice_bits)

Alice bits  [1 1 0 0 1 1 1 1 1 0]


In [63]:
## Step 2
# Alice crea un array di basi di misura (0 = Z ; 1 = X)
alice_basi = randint(2, size = n)

# Stampa Alice basi
print('Alice basi:', end = ' ')
for i in range(n):
    if alice_basi[i] == 0:
        print('Z', end = ' ')
    else:
        print('X', end = ' ')
        
# Crea messaggio tramite la funzione definita 'codifica_messaggio'
messaggio = codifica_messaggio(alice_bits, alice_basi)

# Stampa i circuiti relativi a ogni qubit
print()
print()
print('Circuiti relativi a ogni qubit')
print()
for i in range(n):
    print('Qubit', i, '- valore', alice_bits[i], '- base', end = ' ')
    
    if alice_basi[i] == 0:
        print('Z')
    else:
        print('X')
    
    print(messaggio[i])
    print()

Alice basi: Z X Z X X Z Z X Z Z 

Circuiti relativi a ogni qubit

Qubit 0 - valore 1 - base Z
     ┌───┐ ░ 
q_0: ┤ X ├─░─
     └───┘ ░ 
c: 1/════════
             

Qubit 1 - valore 1 - base X
     ┌───┐┌───┐ ░ 
q_0: ┤ X ├┤ H ├─░─
     └───┘└───┘ ░ 
c: 1/═════════════
                  

Qubit 2 - valore 0 - base Z
      ░ 
q_0: ─░─
      ░ 
c: 1/═══
        

Qubit 3 - valore 0 - base X
     ┌───┐ ░ 
q_0: ┤ H ├─░─
     └───┘ ░ 
c: 1/════════
             

Qubit 4 - valore 1 - base X
     ┌───┐┌───┐ ░ 
q_0: ┤ X ├┤ H ├─░─
     └───┘└───┘ ░ 
c: 1/═════════════
                  

Qubit 5 - valore 1 - base Z
     ┌───┐ ░ 
q_0: ┤ X ├─░─
     └───┘ ░ 
c: 1/════════
             

Qubit 6 - valore 1 - base Z
     ┌───┐ ░ 
q_0: ┤ X ├─░─
     └───┘ ░ 
c: 1/════════
             

Qubit 7 - valore 1 - base X
     ┌───┐┌───┐ ░ 
q_0: ┤ X ├┤ H ├─░─
     └───┘└───┘ ░ 
c: 1/═════════════
                  

Qubit 8 - valore 1 - base Z
     ┌───┐ ░ 
q_0: ┤ X ├─░─
     └───┘ ░ 
c: 1/════════
        

In [64]:
## Step 3
# Bob genera una sequenza di basi
bob_basi = randint(2, size = n)

# Stampa Alice basi
print('Alice basi:                                      ', end = ' ')
for i in range(n):
    if alice_basi[i] == 0:
        print('Z', end = ' ')
    else:
        print('X', end = ' ')

print()
        
# Stampa basi di Bob
print('Bob basi:                                        ', end = ' ')
for i in range(n):
    if bob_basi[i] == 0:
        print('Z', end = ' ')
    else:
        print('X', end = ' ')
print()
print()
print('Alice bits:                                       ', end = '')
for i in alice_bits:
    print(i, end = ' ')
print()

# Bob misura il messaggio di Alice con le sue basi
bob_misura = misura_messaggio(messaggio, bob_basi)

print('Bob misura il messaggio di Alice con le sue basi: ', end = '')
for i in bob_misura:
    print(i, end = ' ')


Alice basi:                                       Z X Z X X Z Z X Z Z 
Bob basi:                                         Z X Z Z X Z Z Z X Z 

Alice bits:                                       1 1 0 0 1 1 1 1 1 0 
Bob misura il messaggio di Alice con le sue basi: 1 1 0 0 1 1 1 0 0 0 

In [65]:
# Alice e Bob scartano entrambi i bit inutili e usano i bit rimanenti per formare le loro chiavi segrete:

## Step 4
alice_chiave = rimozione_differenze(alice_basi, bob_basi, alice_bits)
bob_chiave = rimozione_differenze(alice_basi, bob_basi, bob_misura)
print(alice_chiave)
print(bob_chiave)

[1, 1, 0, 1, 1, 1, 0]
[1, 1, 0, 1, 1, 1, 0]


In [66]:
# np.mod(a,b) calcola il resto della divisione a/b
# v = arr.pop(i) rimuove l'elemento iesimo dall'array arr e lo copia in v 

In [67]:
## Step 5
# randint(a, size=n) genera un array di n numeri random compresi tra 0 e a-1
lung_selezione = 4
bit_selezionati = randint(n, size=lung_selezione)

bob_test = selezione_bits(bob_chiave, bit_selezionati)
print("  bob_test = " + str(bob_test))
alice_test = selezione_bits(alice_chiave, bit_selezionati)
print("alice_test = "+ str(alice_test))

  bob_test = [0, 1, 1, 0]
alice_test = [0, 1, 1, 0]


In [69]:
if bob_test == alice_test:
    print('*** Test chiavi coincidenti ***')
else:
    print('--- Test chiavi non coincidenti ---')
    
print('Alice chiave:', alice_chiave)
print('Bob chiave:  ', bob_chiave)
print("lunghezza chiave = %i" % len(alice_chiave))

*** Test chiavi coincidenti ***
Alice chiave: [1, 1, 1]
Bob chiave:   [1, 1, 1]
lunghezza chiave = 3
